<img src="https://lh3.googleusercontent.com/pw/ACtC-3fFHZrzKpHGWl0vYz7Sr8FX8QqLQ_tc8XHBSwqQnM4hgsIOjtjaOde1M9oHSAfe1Fs2SwVORlapit4-JOz0mjP8Tnz6HetkLZDZb8CifSd0uoSp1Nj3wG_wh1sEQlKXXzvEA9Y9HnQqu2Ecv2igmInb=w1097-h235-no?authuser=0" alt="2020年度ゲノム情報解析入門" height="100px" align="middle">

# データ解析体験(2) - ゲノム解析とディープラーニング -

## ディープラーニングの歴史
<small>引用元: [話題のディープラーニングとは｜過去の歴史とあわせて仕組みを解説](https://service.plan-b.co.jp/blog/dmp/6210/)</small>

**（1943年）「形式ニューロン」の開発**

<img src = "https://service.plan-b.co.jp/wp/wp-content/uploads/blog/deep_learning_01.jpg" alt = "filter" height = "250px">

例えばある形式ニューロンを、「A君が学校に行くかどうかを判定する分類器」としたとします。A君が学校に行ったとき、行かなかったときのあらゆる情報を集め、この形式ニューロンに学習させる事によって以下のように重みと閾値が決定されます。


<img src = "https://service.plan-b.co.jp/wp/wp-content/uploads/blog/deep_learning_03z.jpg" alt = "filter" height = "250px">

**（1958年）「パーセプトロン」の開発**

形式ニューロンをいくつか並列に組み合わせてから出力ニューロンで束ねるという2層の構造をとる「パーセプトロン」が開発されました。


<img src = "https://service.plan-b.co.jp/wp/wp-content/uploads/blog/deep_learning_04.jpg" alt = "filter" height = "250px">

**（1986年）「隠れ層」の導入**

間に隠れ層と呼ばれるもう一つのニューロンの層を加える事で、線形分離不可能な複雑な問題も扱える様になりました。


<img src = "https://service.plan-b.co.jp/wp/wp-content/uploads/blog/deep_learning_06.jpg" alt = "filter" height = "250px">

**（2006年）「ディープラーニング」の開発**

「ニューラルネットワークの層の数を増やしてもうまく学習する方法」が編み出され、これがディープラーニングと呼ばれる様になりました。


<img src = "https://service.plan-b.co.jp/wp/wp-content/uploads/blog/deep_learning_07.jpg" alt = "filter" height = "250px">


## ゲノム解析にディープラーニングを用いる例
<small>引用元: [7. 実践編: ディープラーニングを使った配列解析](https://japan-medical-ai.github.io/medical-ai-course-materials/notebooks/07_DNA_Sequence_Data_Analysis.html)</small>


次世代シーケンサの発展・普及とともに，大量の遺伝子配列が読み取られるようになりました．そうした中で，塩基配列で表現された遺伝子型と病気や形態などの表現型との関係を推定するようなGWAS（Genome Wide Association Study; ゲノムワイド関連解析）が広がってきました．

しかし，遺伝子の変異だけでは全ての表現型の変化を説明できないことがわかってきました．特に，非翻訳領域が遺伝子発現に影響を与え，表現型の変化を生じさせていることが様々な実験結果からわかってきています．遺伝子発現時に周辺領域がどのように影響を与えているのかを調べるためにディープラーニングを利用できるのではないでしょうか。

**扱うデータ**

数百種類の人の細胞型から得られた数千のChIP-seq，DNase-seq（オープンクロマチン領域の網羅的解析の一手法）のデータセットから得られたDNA塩基配列を入力として，CAGE（正確性、定量性に優れたRNAシーケンス手法）の結果計測されたmRNAの発現量を推定する問題を考えます。

****
<<学習用>>

長さが130172bpの配列が5000個あり、それぞれA, T, C, Gの対応する次元の値が1, それ以外は0であるような配列

train_in (5000, 131072, 4)

長さが1024bpからなる配列が5000個あり,それぞれが10種類の異なるChIP-seqの結果のカバレッジ値

train_out (5000, 1024, 10)

****

<<検証用>>


valid_in (500, 131072, 4)

valid_out (500, 1024, 10)

****

<<テスト用>>


test_in (500, 131072, 4)

test_out (500, 1024, 10)

****


**モデルとモデルの学習**

モデルには配列データを1次元の画像とみなし，画像処理の時と同様な解析を行えるCNN（畳み込みニューラルネットワーク）をモデルとして使います。


ただし、


・従来のCNN

ある位置の入力の情報は各層で**隣接する位置**からしか読み込まれません。今回の問題の場合必要な層数が多くなりすぎてしまい現実的ではありません。

<img src = "http://musyoku.github.io/images/post/2016-09-17/naive_conv.png" alt = "filter" height = "250px">

・Dilated Convolution（atrous convolutionやconvolution weith holesともよばれます)

入力の情報を**離れたところ**からも受け取れる（少ない層で学習できる）のでこちらを学習すべきニューラルネットワークとして採用します。

<img src = "https://storage.googleapis.com/deepmind-live-cms/documents/BlogPost-Fig2-Anim-160908-r01.gif" alt = "filter" height = "250px">


〜この後の流れ〜

**学習用データと検証用データでニューラルネットワークの学習**

（参考にしたサイトでは**Chainer**ライブラリを使ってディープラーニングを行っています）

**テストデータに対して学習済みのモデルを適用**

↓にテストデータを使った検証結果の一部を表示します。ピークをきちんと捉えており、配列データを使ってmRNAの発現量を推定することができました。

<img src = "https://japan-medical-ai.github.io/medical-ai-course-materials/_images/notebooks_07_DNA_Sequence_Data_Analysis_43_1.png" alt = "filter" height = "400px">



****

シーケンスデータソース：Sequential regulatory activity prediction across chromosomes with convolutional neural networks, D. R. Kelly and et al., Genome Res. 2018. 28: 739-750
https://genome.cshlp.org/content/28/5/739.full


## Keras/Tensorflow で手書き文字の画像分類

MNISTは手書きの数字を白黒画像にしたデータです。

 ここでは人工知能ライブラリ Keras で MNIST データを学習させ、手書き数字を認識できる**人工知能**を作ります。



Kerasを使用すると、ディープラーニングのベースとなっている数学的理論の部分をゼロから開発せずとも、比較的**短い**ソースコードで実装することができます。

## 本日実現したいこと (作成する人工知能の能力)
今回利用する MNIST のデータは、(1)次のような手書きの 0 〜 9 の数字が書かれた画像データ と、(2)その画像に書かれた実際の数字データ のペアです。

<img src =　https://weblabo.oscasierra.net/wp-content/uploads/2017/08/python-ai-mnist-data-detail-1.png alt = sample>

### **入力データ**
前述の通り、入力データは1つの数字が書かれた画像データです。 MNISTの画像1つは 28×28 ピクセル(＝784ピクセル)の大きさです。 1ピクセルごとに「白」〜「黒」までの色情報が記録されています。 1つの画像の全てのピクセルの情報を人工知能の入力としますので、入力の数は784個になります。

### **出力データ**
画像データを入力された人工知能は、どの数字の画像なのかを考えたあとに、「0」〜「9」までの数字毎にその確率を出力します。 例えば次の表のようなイメージです。 次の表の例だと「3」の確率が一番高いので、人工知能が「3」だと判断したことになります。 このように数字ごとに確率を出力するので、出力の数は10個になります。

<table>
<thead><tr>
<th style="text-align:left">数字</th>
<th style="text-align:left">確率</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left">0</td>
<td style="text-align:left">0.014</td>
</tr>
<tr>
<td style="text-align:left">1</td>
<td style="text-align:left">0.001</td>
</tr>
<tr>
<td style="text-align:left">2</td>
<td style="text-align:left">0.013</td>
</tr>
<tr>
<td style="text-align:left">3</td>
<td style="text-align:left">0.719</td>
</tr>
<tr>
<td style="text-align:left">4</td>
<td style="text-align:left">0.034</td>
</tr>
<tr>
<td style="text-align:left">5</td>
<td style="text-align:left">0.016</td>
</tr>
<tr>
<td style="text-align:left">6</td>
<td style="text-align:left">0.023</td>
</tr>
<tr>
<td style="text-align:left">7</td>
<td style="text-align:left">0.065</td>
</tr>
<tr>
<td style="text-align:left">8</td>
<td style="text-align:left">0.086</td>
</tr>
<tr>
<td style="text-align:left">9</td>
<td style="text-align:left">0.029</td>
</tr>
</tbody>
</table>


## 1.CSVファイルの作成

モデルの学習結果を保存しておけるCSVファイルを作成します。後に学習結果をグラフで描画する際に活用します。
（この手順は任意です。変数に代入してメモリ上に保持しておくこともできます。）


In [ ]:
import os
import pathlib                            #ファイル操作が得意なpathlibモジュールのインポート

CSV_FILE_PATH = "trainlog.csv"            #モデルのトレーニング結果を載せるcsvファイルの設定

if not os.path.exists(CSV_FILE_PATH):     #ファイルが存在しないなら
    pathlib.Path(CSV_FILE_PATH).touch()   #ファイル作成

## 2.データの読み込み

Tensorflowから実行環境(このファイル)へMNIST Datasetを読み込みましょう．
TensorflowはGoogleが開発したオープンソース機械学習プラットフォームです。

In [ ]:
import tensorflow as tf

#tensorflowのKerasからMNISTデータの取得
mnist = tf.keras.datasets.mnist

#　MNISTデータからトレーニング用データとテスト用データ作成
(X_train, y_train),(X_test, y_test) = mnist.load_data()

MNISTデータは、次の4つのファイルで構成されています。

t10k-images-idx3-ubyte.gz:検証用の画像セット


t10k-labels-idx1-ubyte.gz:検証用のラベルセット


train-images-idx3-ubyte.gz:学習用の画像セット


train-labels-idx1-ubyte.gz:学習用のラベルセット

## 3.メモリ節約（小技）

MNISTの生データが入っているmnist変数はもう不要なのでメモリから完全に削除しましょう

In [ ]:
del mnist          #不要な変数mnistを削除


#以下の操作はpythonでは自動的に行ってくれるが、今回は明示的に行ってみる
import gc          #gcモジュールをインポート
gc.collect()       #不要になったメモリ領域を解放して再利用可能にする

## 4.MNIST Datasetのサンプルデータを見てみる


データセットの型はnumpy.ndarrayです。

最初にshape関数で配列の形状を取得して確認してみましょう

中身は学習用データは数字の画像6万枚分、検証用データは１万枚分のデータとなっていて、
さらに画像データは28x28ピクセルで一つの数字を表しているのでその内容が反映された結果となるはずです。

In [ ]:
print("X_train : ", X_train.shape)
print("y_train : ", y_train.shape)
print("X_test : ", X_test.shape)
print("y_test : ", y_test.shape)

次に実際に学習用データの
X(白黒画像)とy(ラベル)を視覚的に確認してみましょう

（検証用データも同様な中身になっています）

In [ ]:
import matplotlib
import matplotlib.pyplot as plt                      #画像描画用のライブラリにmatplotlibを使います。
%matplotlib inline

#1, 10, 100番目のデータ　について確認したい
for i in [1,10,100]:
    print("y_train", "(i="+str(i)+"): ", y_train[i])     #　i番目のラベル表示
    print("X_train", "(i="+str(i)+"): ")
    plt.imshow(X_train[i], cmap='gray')              #matplotliibを使いグレースケールで数字の画像表示
    plt.show()

## 5.画像データの正規化
全ての特徴の大きさを同じレベルで扱うために、学習モデルを構築する前の前処理としてデータを正規化（０〜１の大きさに限定）する必要があります。

代表的な正規化手法としては以下2つが挙げられます．

### **min-max normalization**
最小値が0, 最大値が1になるように変換する


x_new = (x - x_min) / (x_max - x_min)


### **z-score normalization**


平均が0, 標準偏差が1になるよう変換する


x_new = (x - x_mean) / x_std

---------------------------------------------------

↓で確認できるように、MNIST Datasetに含まれる画像データでは，各画素の値が「0以上255以下」の8bit整数で表現されています．

In [ ]:
print("X_train min", X_train.min())
print("X_train max", X_train.max())

今回は，これにmin-max normalizationを適用することで，値の範囲を「0~1」に限定させます．

In [ ]:
# Min-Max Normalization
X_train, X_test = X_train/255.0, X_test/255.0

上手く正規化できたか確認します

In [ ]:
print("X_train min", X_train.min())
print("X_train max", X_train.max())

## 6.CNNモデルの作成


<img src = "https://kenyu-life.com/wp-content/uploads/2019/03/%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%B3%E3%82%B7%E3%83%A7%E3%83%83%E3%83%88-2019-03-07-8.56.07.jpg" alt = "filter" height = "250px">

<small>画像は[kenyu-life.com](https://kenyu-life.com/2019/03/07/convolutional_neural_network/) から引用 </small>

CNN（畳み込みニューラルネットワーク）では、


1. 畳み込みフィルタ層：画像の濃淡パターンを検出します。（エッジ抽出等の特徴抽出）

<img src = "https://kenyu-life.com/wp-content/uploads/2019/03/3cnn.gif" alt = "filter" height = "400px">

<small>画像は[kenyu-life.com](https://kenyu-life.com/2019/03/07/convolutional_neural_network/) から引用 </small>

2. プーリング層：物体の位置が変動しても同一の物体であると見なせるようにします。（位置ズレを許容する）また，プーリングには，「maxプーリング」と「avgプーリング」の2種類があります．

<img src = "https://kenyu-life.com/wp-content/uploads/2019/03/pooling.gif" alt = "filter" height = "200px">

<small>画像は[kenyu-life.com](https://kenyu-life.com/2019/03/07/convolutional_neural_network/) から引用 </small>

　これらの層を組み合わせることによって、画像から特徴量を抽出する働きを担っています。


　一方、特徴量を抽出するだけでは、画像の識別はできません。識別には、「特徴量に基づいた分類」が必要です。この分類の役割を担っているのが、全結合層と出力層です。


3. 全結合層：①②を通して特徴部分が取り出された画像データを一つのノードに結合し、活性化関数（後述）によって変換された値（特徴変数）を出力します。ノードの数が増えると特徴量空間の分割数が増し、各領域を特徴付ける特徴変数の数が増えます。

4. 出力層：全結合層からの出力（特徴変数）を元に、ソフトマックス関数を用いて確率に変換し、それぞれの領域に正しく分類される確率を最大化する（最尤推定法）ことによって分類を行います。



In [ ]:
model = tf.keras.models.Sequential([
    # (None, 28, 28) -> (None, 784)
    tf.keras.layers.Flatten(input_shape=(28, 28), name='input'),
    
    # Layer1: 線形写像（一次変換）: (None, 784) -> (None, 512)
    tf.keras.layers.Dense(512, name='fc_1'),
    # 活性化関数: ReLU　　（活性化関数とは入力のなんらかの合計（しばしば、線形な重み付け総和）から、出力を決定するための関数）
    tf.keras.layers.Activation(tf.nn.relu, name='relu_1'),
    
    # Layer2: 線形写像（一次変換）: (None, 512) -> (None, 256)
    tf.keras.layers.Dense(256, name='fc_2'),
    # 活性化関数: ReLU
    tf.keras.layers.Activation(tf.nn.relu, name='relu_2'),
    
    # Layer3: 線形写像（一次変換）: (None, 256) -> (None, 256)
    tf.keras.layers.Dense(256, name='fc_3'),
    # 活性化関数: ReLU
    tf.keras.layers.Activation(tf.nn.relu, name='relu_3'),
    
    # Layer4: 線形写像（一次変換）: (None, 256) -> (None, 10)
    tf.keras.layers.Dense(10, name='dense_3'),
    # 活性化関数: Softmax
    tf.keras.layers.Activation(tf.nn.softmax, name='softmax')
])

In [ ]:
# 作成したモデルの構造を確認します
model.summary()

## 7.コンパイル

モデルを使った学習の前に、どのような学習処理を行うかをcompileメソッドを用いて設定します。


In [ ]:
# モデルと学習の情報をメモリにセット (CPU or GPU)
model.compile(optimizer='adam',                         #最適化アルゴリズム。adamは定番
              loss='sparse_categorical_crossentropy',   #ズレの大きさを定量化する損失関数
              metrics=['accuracy'])                     #訓練やテストの際にモデルを評価するための評価関数のリスト

＊損失とはニューラルネットワークによる予測値と正解値との**ズレ**のことです。そのため、ニューラルネットワークの予測値が変われば、損失も変わります。そのニューラルネットワークの予測値はパラメータ(重みおよびバイアス)で変化します。損失は小さいほどいいです

## 8.寄り道-コールバック関数の設定

モデルの学習途中記録を可視化するためにコールバック関数（トレーニング中にコールされる）を設定します。

In [ ]:
callbacks = []
callbacks.append(tf.keras.callbacks.CSVLogger(CSV_FILE_PATH))

## 9.モデルの学習

In [ ]:
history = model.fit(X_train, y_train,  #学習様データを使いながら、KerasライブラリのModelクラスのfit関数で学習します
                    batch_size=100,  #　データを１００個ずつに分ける。データサイズは６００００なので全部で６００回の学習になる
                    epochs=30, #　学習反復回数
                    verbose=1,  #ログの詳細表示オプション。1の場合はログをプログレスバーで標準出力
                    validation_data=(X_test, y_test), #検証データで検証（今回はテストデータで検証してます）
                    callbacks=callbacks)

## 10.学習済みモデルの評価
学習済みモデル(model)のtestデータ(X_test,y_test)に対するの正答率(accuracy)と損失関数の値(loss)を確認します。

In [ ]:
train_loss, train_acc = model.evaluate(X_train, y_train, verbose=1)
print("loss(train): {:.4}".format(train_loss))
print("accuracy(train): {:.4}".format(train_acc))

print()

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print("loss(test): {:.4}".format(test_loss))
print("accuracy(test): {:.4}".format(test_acc))

用意しておいたCSVファイルを元に，学習曲線(モデルに対する評価指標の経過を表す)を描画してみます。まずは中身を確認します。

In [ ]:
import pandas as pd 
df = pd.read_csv(CSV_FILE_PATH)
df.head()

グラフ描画に使う変数を設定します

In [ ]:
epochs = df["epoch"].values
train_acc = df["accuracy"].values
train_loss = df["loss"].values
test_acc = df["val_accuracy"].values
test_loss = df["val_loss"].values

###**損失関数の値**

In [ ]:
plt.plot(epochs, train_loss, label="train data")#　x座標データ　y座標データ　ラベル
plt.plot(epochs, test_loss, label="test data")
plt.xlabel("epochs")#　x軸ラベル
plt.ylabel("loss\n(categorical crossentropy)")#　y軸ラベル
plt.legend(loc="upper right")#　右上に凡例
plt.show()

###**画像分類の正答率**

学習用データの曲線は、1.00にかなり近いところまでいってます。
検証用データ（今回はテストデータ）の曲線との差はグラフからはかなりあるように見えますが、0.02(2%)程度です。

In [ ]:
plt.plot(epochs, train_acc, label="train data")
plt.plot(epochs, test_acc, label="test data")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.legend(loc="lower right")
plt.show()

因みに学習用データを60000→600に減らすと、局所解に陥り検証データの曲線の精度が１０％以上落ちます。

## 11.学習済みモデルによる推論計算
学習済みモデルを用いて，testデータに対する推論計算を行い，分類結果を見てみます。

In [ ]:
 for i in [0,1,2]:
    y_true = y_test[i]
    y_pred = model.predict_classes(X_test[i].reshape(1,28,28))[0]
    print("モデルによる予測", "(i="+str(i)+"): ", y_pred)
    print("学習済みモデルに流す画像データのラベル", "(i="+str(i)+"): ", y_true)
    print("学習済みモデルに流す画像データ", "(i="+str(i)+"): ")
    plt.imshow(X_test[i], cmap='gray')
    plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 8))

ROW = 4
COLUMN = 5

for i in range(ROW * COLUMN):
    y_true = y_test[i]
    y_pred = model.predict_classes(X_test[i].reshape(1,28,28))[0]
    
    if y_true == y_pred:
        result = "True" # 正解
    else:
        result = "False" # ハズレ
    
    plt.subplot(ROW, COLUMN, i+1)
    plt.imshow(X_test[i], cmap='gray')
    plt.title("No.{} - {}\ny_true:{}, y_pred:{}".format(i, result, y_true, y_pred))
    plt.axis("off")

fig.tight_layout()
fig.show()

## 12.学習済みモデルの保存
Kerasでは，全てのニューラルネットワークモデルがkeras.models.Model()クラスのインスタンスとなっています．学習済みモデルmodelに対して， model.save()を実行することで「モデルの保存」が完了します．

In [ ]:
ins_path = 'trained_model_v0.h5'
model.save(ins_path)

##（発展）ディープラーニング  転移学習

転移学習(Transfer Learning)とは、ある領域で学習したこと(学習済みモデル)を別の領域に役立たせ、効率的に学習させる方法です。

<img src = "https://s3-ap-northeast-1.amazonaws.com/ledge-assets/media/wp-content/uploads/2020/05/08173933/fig1.png" alt = "filter" height = "250px">
出典：https://ledge.ai/transfer-learning/

**実用例：Survival Prediction**

参考：Transfer learning with convolutional neural networks for cancer survival prediction using gene-expression data（Guillermo López-García, José M. Jerez, Leonardo Franco, Francisco J. Veredas）

https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0230536

肺癌を除く３１種類の異なる癌について、ゲノム配列と患者の生存率データを使って**予め学習した**CNNモデルを用意します。
肺癌のRNAシーケンスデータを使ってそのCNNモデルをファインチューニング（学習済みモデルの最終出力層を付け替え、入力層に近い部分と出力層のパラメータも変更する）します。


結果として、ゲノム配列を学習する際のCNNの弱点（隣接する塩基の関連性が見出しづらい等）を克服し、他の機械学習手法（MLNNsや他のML手法）よりも高パフォーマンスな予測ができたという報告がありました。

##（おまけ）ディープラーニング   深層強化学習

**特徴**

1. 教師データが無くても学習できる（環境に対する試行錯誤によってターゲットをサンプリングできる）

1. 未知の環境に対しても学習できる


**例**

  深層強化学習を使ったゲーム攻略

https://qiita.com/yukiB/items/0a3faa759ca5561e12f8

<img src = "https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F59864%2F6dcd9255-4f66-c606-23fd-d0002087ae80.gif?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&w=1400&fit=max&s=8e83c2114104c6626db0d67bfa2d12a0" alt = "filter" height = "250px">
<img src = "https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.amazonaws.com%2F0%2F59864%2F78af77ed-882b-72c1-983b-1ba0036cb048.gif?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&w=1400&fit=max&s=b86f47064a104c8111f794aafe701a6d" alt = "filter" height = "250px">